In [1]:
from __future__ import print_function

import argparse
import os
import sys
import time
import scipy.io as sio
from PIL import Image

import tensorflow as tf
import numpy as np

from model import DeepLabResNetModel


def load(saver, sess, ckpt_path):
    saver.restore(sess, ckpt_path)
    print("Restored model parameters from {}".format(ckpt_path))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
IMG_MEAN = np.array((104.00698793,116.66876762,122.67891434), dtype=np.float32)

img_path = "input/paper2.jpg"
filename = img_path.split('/')[-1]
file_type = filename.split('.')[-1]
restore_from = "restore_weights"

# testing if image file exists
if os.path.isfile(img_path):
    print('successful load img: {0}'.format(img_path))
else:
    print('not found file: {0}'.format(img_path))
    sys.exit(0)

# Prepare image.
if file_type.lower() == 'png':
    img = tf.image.decode_png(tf.read_file(img_path), channels=3)
elif file_type.lower() == 'jpg':
    img = tf.image.decode_jpeg(tf.read_file(img_path), channels=3)
else:
    print('cannot process {0} file.'.format(file_type))

# Convert RGB to BGR.
img_r, img_g, img_b = tf.split(axis=2, num_or_size_splits=3, value=img)
img = tf.cast(tf.concat(axis=2, values=[img_b, img_g, img_r]), dtype=tf.float32)
# Extract mean.
img -= IMG_MEAN

successful load img: input/paper2.jpg


In [3]:
NUM_CLASSES = 27

# Create network.
net = DeepLabResNetModel({'data': tf.expand_dims(img, dim=0)}, is_training=False, num_classes=NUM_CLASSES)

# Which variables to load.
restore_var = tf.global_variables()

# Predictions.
raw_output = net.layers['fc_out']
raw_output_up = tf.image.resize_bilinear(raw_output, tf.shape(img)[0:2,])
raw_output_up = tf.argmax(raw_output_up, dimension=3)
pred = tf.expand_dims(raw_output_up, dim=3)

# Set up TF session and initialize variables.
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()

sess.run(init)

# Load weights.
ckpt = tf.train.get_checkpoint_state(restore_from)

if ckpt and ckpt.model_checkpoint_path:
    loader = tf.train.Saver(var_list=restore_var)
    load_step = int(os.path.basename(ckpt.model_checkpoint_path).split('-')[1])
    load(loader, sess, ckpt.model_checkpoint_path)
else:
    print('No checkpoint file found.')
    load_step = 0

Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from restore_weights\model.ckpt-100000
Restored model parameters from restore_weights\model.ckpt-100000


In [4]:
# Perform inference.
preds = sess.run(pred)

In [ ]:
# Save the trained model

classification_inputs = tf.saved_model.utils.build_tensor_info(pred)
classification_output_classes = tf.saved_model.utils.build_tensor_info(pred)

classification_signature = ( tf.saved_model.signature_def_utils.build_signature_def (
    inputs  = { tf.saved_model.signature_constants.CLASSIFY_INPUTS         : classification_inputs },
    outputs = { tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES : classification_output_classes },
    method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME ) )


builder = tf.saved_model.builder.SavedModelBuilder( "message-classification-model-dir2" )
builder.add_meta_graph_and_variables(
      sess, [ tf.saved_model.tag_constants.SERVING ],
      signature_def_map={ "classify_message" : classification_signature, },
      main_op=tf.tables_initializer() )
builder.save()

!tar -zcvf message-classification-model.tar.gz -C message-classification-model-dir2 .

In [ ]:
# Create a Watson Machine Learning client instance
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials = {
    "apikey"    : "Add here",
    "instance_id" : "Add here,
    "url"    : "Add here"
}
client = WatsonMachineLearningAPIClient( wml_credentials )

In [ ]:
# Getting model ready for deployment

metadata = {
    client.repository.ModelMetaNames.NAME: "TensorFlow model",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.14"
}
model_details_targz = client.repository.store_model( model="message-classification-model.tar.gz", meta_props=metadata )

In [ ]:
# Deploy the stored model as an online web service deployment

model_id_dir = model_details_targz["metadata"]["guid"]
deployment_details_dir = client.deployments.create( artifact_uid=model_id_dir, name="TensorFlow deployment" )